Intraday - RSI strategy



In [1]:
import pandas_ta as ta
import requests
import time 
import os

from datetime import datetime, timedelta
from positions import PositionSide, TradeSide
from dotenv import load_dotenv

# All the Alpaca Loaders
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest, StockBarsRequest, CryptoLatestBarRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.trading.models import Order
from alpaca.data.models.bars import Bar
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

crypto_data_client = CryptoHistoricalDataClient()

In [2]:
days_back = 1
now = datetime.now()
start_time = now - timedelta(days = days_back)
timeframe = TimeFrame(amount = 5, unit = TimeFrameUnit.Minute)
data_request = CryptoBarsRequest(
    symbol_or_symbols="ETH/USD", 
    end = now, 
    limit = 100, 
    timeframe = timeframe
)

df = crypto_data_client.get_crypto_bars(request_params = data_request).df

In [3]:
rsi_value = ta.rsi(close = df['vwap']).dropna().iloc[-1]

In [7]:
class CryptoLongRSIStrategy: 

    def __init__(self, symbol: str, timeframe: TimeFrame, api_key: str, secret_key: str, capital: float, entry_rsi_value: int = 30, exit_rsi_value: int = 70, rsi_lookback_period: int = 14) -> None:
        self.symbol = symbol
        self.capital = capital
        self.entry_rsi_value = entry_rsi_value
        self.exit_rsi_value = exit_rsi_value
        self.timeframe = timeframe
        self.rsi_lookback_period = rsi_lookback_period
        self.__api_key = api_key 
        self.__secret_key = secret_key

        self.trading_client = TradingClient(self.__api_key, self.__secret_key)
        self.crypto_data_client = CryptoHistoricalDataClient(api_key = self.__api_key, secret_key = self.__secret_key)
        self.telegram_token = '5781019360:AAHHE8_MV4qpgZE6U56o-otnhwZ2kVevNYQ'
        self.telegram_chat_id = 1634990243
        self.trades = []



    def get_historical_data(self, column: str = 'vwap'): 
        columns = ['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap']
        assert column in columns, "Column passed must be from open, high, low, close, volume, trade_count, or vwap"

        data_request = CryptoBarsRequest(
            symbol_or_symbols=self.symbol, 
            end = datetime.now(), 
            limit = self.rsi_lookback_period + 10, 
            timeframe = self.timeframe
        )

        data = self.crypto_data_client.get_crypto_bars(request_params=data_request).df

        return data[column]
    
    
    
    def send_sell_market_order(self, quantity: float) -> Order:
        sell_order_params = MarketOrderRequest(
            symbol = self.symbol, 
            qty = quantity, 
            side = OrderSide.SELL, 
            time_in_force = TimeInForce.GTC
        ) 

        sell_order = self.trading_client.submit_order(sell_order_params)
        time.sleep(0.5)
        sell_order_update = self.trading_client.get_order_by_client_id(sell_order.id)
        return sell_order_update
    
    def send_buy_market_order(self, quantity: float) -> Order: 
        buy_order_params = MarketOrderRequest(
            symbol = self.symbol, 
            qty = quantity, 
            side = OrderSide.BUY, 
            time_in_force = TimeInForce.GTC
        ) 

        buy_order = self.trading_client.submit_order(buy_order_params)
        time.sleep(0.5)
        buy_order_update = self.trading_client.get_order_by_client_id(buy_order.id)
        return buy_order_update
    
    def send_long_market_order(self) -> Order: 
        quantity = self.get_shares_from_capital()
        order = self.send_buy_market_order(quantity=quantity)
        return order
    
    def get_latest_bar(self) -> Bar: 

        latest_bar_request = CryptoLatestBarRequest(
            symbol_or_symbols=  'ETH/USD'
        )

        latest_bar = crypto_data_client.get_crypto_latest_bar(request_params=latest_bar_request)
        return latest_bar[self.symbol]
    
    def get_shares_from_capital(self): 
        last_bar = self.get_latest_bar()
        total_shares = self.capital / last_bar.vwap

        return total_shares
    
    def generate_signal(self): 
        historical_data = self.get_historical_data()

        rsi = ta.rsi(close = historical_data, length = self.rsi_lookback_period).iloc[-1]

        if rsi <= self.entry_rsi_value: 
            return "LONG"
        elif rsi >= self.exit_rsi_value: 
            return "EXIT"
        else: 
            return "NEUTRAL"
    
    def send_telegram_message(self, message): 
        url = f"https://api.telegram.org/bot{self.telegram_token}/sendMessage?chat_id={self.telegram_chat_id}&text={message}"

        try:
            response = requests.get(url)

            if not response:
                print(response.status_code)

        except Exception as e:
            print(e)

    def get_position(self) -> dict: 
        """Constructs our cumulative position from a set of trades

        Returns:
            dict: Dictionary of our position with the keys "side" and "quantity". The values for side are "LONG", "SHORT", "NEUTRAL"
        """ 
        traded_quantity = 0

        if len(self.trades) != 0: 

            # We go through each trade and do a running total
            for trade in self.trades: 
                traded_quantity += trade.qty * trade.side

        if traded_quantity > 0: 
            side = PositionSide.LONG
        elif traded_quantity < 0: 
            side = PositionSide.SHORT
        else: 
            side = PositionSide.NEUTRAL

        return {"side": side, "quantity": traded_quantity}
            

    def run(self): 

        self.send_telegram_message(message = "Running the RSI strategy")
        signal = self.generate_signal()
        current_position = self.get_position() 
        total_quantity = self.get_shares_from_capital() 

        if signal == 'LONG': 

            self.send_telegram_message(f"RSI is below {self.entry_rsi_value}, we should go long")

            # Check our positions to see if we are long already

            if current_position['side'] in [PositionSide.NEUTRAL, PositionSide.SHORT]: 
                if current_position['side'] == PositionSide.NEUTRAL: 
                    self.send_telegram_message("We are neutral right now, going long....")
                    market_order = self.send_long_market_order()
                    self.trades.append(market_order)
                    self.send_telegram_message("Trade successful, we are going long.")

                elif current_position['side'] == PositionSide.SHORT: 
                    # Closing the position down first
                    close_out_short_order = self.send_buy_market_order(quantity = -current_position['side'])
                    self.trades.append(close_out_short_order)

                    # Then we go long
                    long_order = self.send_long_market_order()
                    self.trades.append(long_order)

                
        elif signal == "EXIT": 

            if current_position['side'] == PositionSide.LONG: 
                position_close_out = self.send_sell_market_order(quantity = current_position['quantity'])
                self.trades.append(position_close_out)

        else: 

            self.send_telegram_message("Nothing to do right now")

In [8]:
load_dotenv(dotenv_path=r'C:\Users\srerr\Documents\Projects\PersonalProjects\stonks\alpaca\.env')
symbol = "ETH/USD"
timeframe = TimeFrame(amount = 5, unit = TimeFrameUnit.Minute)
api_key = os.environ['ALPACA_PAPER_TRADING_KEY_ID']
secret_key = os.environ['ALPACA_PAPER_TRADING_SECRET_KEY']

strat = CryptoLongRSIStrategy(
    symbol=symbol, 
    timeframe=timeframe, 
    api_key=api_key, 
    secret_key=secret_key, 
    capital = 200, 
    entry_rsi_value=30, 
    exit_rsi_value=70, 
    rsi_lookback_period=25
)

In [9]:
order = strat.send_buy_market_order(quantity = 3)

In [11]:
order.filled_avg_price

'1167.2'